In [1]:
import pandas as pd
import os

In [2]:
os.chdir(r'C:\Users\user\Documents\GitHub\COVID-19\python\scripts')
df_file = r'C:\Users\user\Documents\GitHub\COVID-19\consolidated_data\country_report.json'

config = pd.read_csv('config.csv',index_col='var').fillna('-')
file_dir = config.loc['flourish_data_dir'].path

data = pd.read_json(df_file)

# region mapping dictionary:
region_mapping_dict = pd.read_csv('region_mapping.csv',header=None,index_col=0).to_dict()[1]

# DATA FOR FLOURISH @ https://app.flourish.studio/@psycho.presley

In [5]:
def flourish_racing_bars(df,parameters,initial_date,file_dir,file_name='racing_bars'):
    '''
    With this function it is possible to generate the dataset as used
    by Florish Studio @ https://app.flourish.studio/@psycho.presley

    Parameters
    ----------
    df: obj, DataFrame
        pandas DataFrame with the data to be used. The DataFrame must
        have been generated by the world_data_formatter function 
        presented in the pycovidfunc.py module
    
    parameters: str, array-like
        list with the columns of df to be used. Each column will
        generate one separate and independent file to be used in
        Flourish studio
    
    initial_date: str
        string of the date in the YYYY-MM-DD format to be the first
        date to be considered in the final file
    
    file_dir: str
        string of the root dir where the flourish data must be saved

    file_name: str
        the name of the *.csv file to be created
    '''
    from pandas import DataFrame, concat
    from os import path

    print('--------------------------')
    print('Creating files for the flourish racing bars chart')
    try:
        countries = df['Country/Region'].unique().tolist()
        
        # The entry 'French Guiana' is resulting in errors in Fourish studio
        # so it will be removed:
        countries.remove('French Guiana')
        
        for item in parameters:
            print('creating the {} cases file'.format(item))
            columns = ['Country/Region','Date',item]
            flourish = DataFrame()

            for country in countries:
                df_aux = df[columns].loc[df['Country/Region'] == country]
                df_aux = df_aux.pivot(index='Country/Region',columns='Date', values=item)

                flourish = concat([flourish,df_aux]).interpolate(method='linear',limit=3)
                flourish.fillna(method='bfill',inplace=True)

            file = path.join(file_dir,file_name + '_' + item + '.csv')
            flourish.loc[:,initial_date:].to_csv(file)
        print('Files created succesfully!')
    except:
        print('Process aborted! No files for flourish studio were created.')
    finally:
        print('End execution of the flourish racing bars chart function.')
        print('--------------------------')                        

In [4]:
def flourish_parliament_map(df,seats,region_mapping_dict,file_dir,places=1000,file_name='parliament_map'):
    '''
    With this function it is possible to generate the dataset as used
    by the parliament map viz in Florish Studio
    @ https://app.flourish.studio/@psycho.presley

    Parameters
    ----------
    df: obj, DataFrame
        pandas DataFrame with the data to be used. The DataFrame must
        have been generated by the world_data_formatter function 
        presented in the pycovidfunc.py module
    
    seats: str, array-like
        list with the columns of df to be used as seats. Each column
        represents one seat tab in the Flourish studio parliament chart
    
    region_mapping_chart: dict
        dictionary with the countries as keys and their region as values
        for region mapping

    file_dir: str
        string of the root dir where the flourish data must be saved
    
    places: int
        desired number of places in the parliament chart

    file_name: str
        the name of the *.csv file to be created
    '''
    from os import path

    print('--------------------------')
    print('Creating files for the flourish studio parliament map')
    try:
        columns = ['Country/Region']
        columns.extend(seats)
        df_aux = df[columns].loc[df['Date'] == max(df['Date'])]

        for item in seats:
            df_aux[item] = df_aux[item].apply(lambda x:places*x/df_aux[item].sum())
        
        # Saving the first file for the countries parliament chart:
        df_aux.to_csv(path.join(file_dir,file_name + '_country.csv'),index=False)
        
        # Now ready to create the regions parliament chart
        # mapping the country -> region:
        df_aux['Country/Region'] = df_aux['Country/Region'].transform(lambda x: region_mapping_dict[x] 
                                                                      if x in region_mapping_dict.keys()
                                                                      else x)
        
        df_aux = df_aux.groupby('Country/Region').sum().reset_index()
        df_aux.to_csv(path.join(file_dir,file_name + '_region.csv'),index=False)

        print('Files created succesfully!')
    except:
        print('Process aborted! No files for flourish studio were created.')
    finally:
        print('End execution of the flourish parliament map function.')
        print('--------------------------')                

In [5]:
def flourish_hierarchy_chart(df,cases,region_mapping_dict,file_dir,file_name='hierarchy_chart'):
    '''
    With this function it is possible to generate the dataset as used
    by the parliament map viz in Florish Studio
    @ https://app.flourish.studio/@psycho.presley

    Parameters
    ----------
    df: obj, DataFrame
        pandas DataFrame with the data to be used. The DataFrame must
        have been generated by the world_data_formatter function 
        presented in the pycovidfunc.py module
    
    cases: str, array-like
        list with the columns of df to be used as seats. Each column
        represents one seat tab in the Flourish studio parliament chart
    
    region_mapping_chart: dict
        dictionary with the countries as keys and their region as values
        for region mapping

    file_dir: str
        string of the root dir where the flourish data must be saved
    
    file_name: str
        the name of the *.csv file to be created
    '''
    from os import path

    print('--------------------------')
    print('Creating files for the flourish studio hierarchy chart')
    try:
        columns = ['Country/Region']
        columns.extend(cases)
        df_aux = df[columns].loc[df['Date'] == max(df['Date'])]

        # mapping the country -> region:
        df_aux['Group'] = df_aux['Country/Region'].transform(lambda x: region_mapping_dict[x] 
                                                                      if x in region_mapping_dict.keys()
                                                                      else x)

        # Saving the first file for the countries parliament chart:
        df_aux.to_csv(path.join(file_dir,file_name + '.csv'),index=False)
        print('Files created succesfully!')
    except:
        print('Process aborted! No files for flourish studio were created.')
    finally:
        print('End execution of the flourish hierarchy chart function.')
        print('--------------------------')        

In [6]:
def flourish_point_map(df,parameters,lat,long,file_dir,file_name='point_map'):
    '''
    With this function it is possible to generate the dataset as used
    by the parliament map viz in Florish Studio
    @ https://app.flourish.studio/@psycho.presley

    Parameters
    ----------
    df: obj, DataFrame
        pandas DataFrame with the data to be used. The DataFrame must
        have been generated by the world_data_formatter function 
        presented in the pycovidfunc.py module
    
    parameters: str, array-like
        list with the columns of df to be used as map variables. Each
        column represents one seat tab in the Flourish studio
        parliament chart
    
    lat: dict
        dictionary with the countries as keys and their latitude
        coordinate as values for mapping
    
    long: dict
        dictionary with the countries as keys and their longitude
        coordinate as values for mapping
    
    file_dir: str
        string of the root dir where the flourish data must be saved
    
    file_name: str
        the name of the *.csv file to be created
    '''
    from os import path
    
    print('--------------------------')
    print('Creating files for the flourish studio point map')
    try:
        df_aux=df[['Country/Region','Date']]

        for item in parameters:
            df_aux = pd.concat([df_aux,
                                df.groupby('Country/Region')[item].diff().fillna(value=0)],
                                axis=1).sort_values(by='Date')

        # mapping the country -> Lat/Long:
        df_aux['Latitude'] = df_aux['Country/Region'].transform(lambda x: lat[x] 
                                                                      if x in lat.keys() 
                                                                      else 0)
        df_aux['Longitude'] = df_aux['Country/Region'].transform(lambda x: long[x] 
                                                                      if x in long.keys() 
                                                                      else 0)

        df_aux.to_csv(path.join(file_dir,file_name + '.csv'),index=False)
        print('Files created succesfully!')
    except:
        print('Process aborted! No files were created.')
    finally:
        print('End execution of the flourish point map function.')
        print('--------------------------')

In [5]:
def flourish_card_plot(df,cases,region_mapping_dict,file_dir,file_name='card_plot'):
    '''
    With this function it is possible to generate the dataset as used
    by the card plot viz in Florish Studio
    @ https://app.flourish.studio/@psycho.presley

    Parameters
    ----------
    df: obj, DataFrame
        pandas DataFrame with the data to be used. The DataFrame must
        have been generated by the world_data_formatter function 
        presented in the pycovidfunc.py module
    
    cases: str, array-like
        list with the columns of df to be used as seats. Each column
        represents one seat tab in the Flourish studio card plot
    
    region_mapping_chart: dict
        dictionary with the countries as keys and their region as values
        for region mapping

    file_dir: str
        string of the root dir where the flourish data must be saved
    
    file_name: str
        the name of the *.csv file to be created
    '''
    from pandas import read_csv
    from os import path
    from quantiphy import Quantity as qty

    print('--------------------------')
    print('Creating files for the flourish studio card plot')
    try:
        columns = ['Country/Region']
        columns.extend(cases)
        df_aux = df[columns].loc[df['Date'] == max(df['Date'])]

        # mapping the country -> region:
        df_aux['Group'] = df_aux['Country/Region'].transform(lambda x: region_mapping_dict[x] 
                                                                      if x in region_mapping_dict.keys()
                                                                      else x)

        df_aux = df_aux.groupby('Group').sum()
        df_aux.drop('Other',inplace=True)

        df_logo = read_csv('region_logo.csv', index_col='Group')
        df_aux = df_aux.join(df_logo, on='Group')

        for item in cases:
            df_aux[item] = df_aux[item].transform(lambda x:qty(x).render(prec=2))

        # Saving the first file for the countries parliament chart:
        df_aux.to_csv(path.join(file_dir,file_name + '.csv'),index=False)
        print('Files created succesfully!')
    except:
        print('Process aborted! No files for flourish studio were created.')
    finally:
        print('End execution of the flourish card plot function.')
        print('--------------------------')        

In [158]:
def flourish_survey_chart(df,cases,region_mapping_dict,file_dir,file_name='survey_chart'):
    '''
    With this function it is possible to generate the dataset as used
    by the survey chart viz in Florish Studio
    @ https://app.flourish.studio/@psycho.presley

    Parameters
    ----------
    df: obj, DataFrame
        pandas DataFrame with the data to be used. The DataFrame must
        have been generated by the world_data_formatter function 
        presented in the pycovidfunc.py module
    
    cases: str, array-like
        list with the columns of df to be used as seats. Each column
        represents one seat tab in the Flourish studio card plot
    
    region_mapping_chart: dict
        dictionary with the countries as keys and their region as values
        for region mapping

    file_dir: str
        string of the root dir where the flourish data must be saved
    
    file_name: str
        the name of the *.csv file to be created
    '''
    from os import path
    from quantiphy import Quantity as qty

    print('--------------------------')
    print('Creating files for the flourish studio survey chart')
    
    # creating quartiles map function:
    def quart_func(x,q,case):
        if x < q[0]:
            return 'very few ' + case.lower() + ' cases'
        elif q[0] <= x < q[1]:
            return 'few ' + case.lower() + ' cases'
        elif q[1] <= x < q[2]:
            return 'high ' + case.lower() + ' cases'
        else:
            return 'very high ' + case.lower() + ' cases'
    
    try:
        columns = ['Country/Region','Confirmed']
        columns.extend(cases)
        df = df[columns].loc[df['Date'] == max(df['Date'])]

        for item in cases:
            new_column = 'percentage of '+item.lower()
            df[new_column] = (df[item]*100/df['Confirmed']).round(2)

            quantile = df[new_column].quantile(q=[0.2,0.5,0.8])
            df[item.lower() + ' cases interval']=df[new_column].apply(
                lambda x:quart_func(x,quantile.values,item)
            )
            #df[new_column] = df[new_column].transform(lambda x:qty(x,'%'))

        # mapping the country -> region:
        df['WHO region'] = df['Country/Region'].transform(lambda x: region_mapping_dict[x]
                                                     if x in region_mapping_dict.keys() 
                                                     else x)
        df.drop(cases,axis=1,inplace=True)

        # Saving the first file for the countries parliament chart:
        df.to_csv(path.join(file_dir,file_name + '.csv'),index=False)
        print('Files created succesfully!')
    except:
        print('Process aborted! No files for flourish studio were created.')
    finally:
        print('End execution of the flourish survey chart function.')
        print('--------------------------')

In [168]:
def flourish_slope_chart(df,file_dir,file_name='slope_chart',
                         case='Confirmed',initial_month=3):
    '''
    With this function it is possible to generate the dataset as used
    by the slope chart viz in Florish Studio
    @ https://app.flourish.studio/@psycho.presley

    Parameters
    ----------
    df: obj, DataFrame
        pandas DataFrame with the data to be used. The DataFrame must
        have been generated by the world_data_formatter function 
        presented in the pycovidfunc.py module
    
    case: str, array-like
        list with the columns of df to be used as seats. Each column
        represents one seat tab in the Flourish studio card plot
    
    region_mapping_chart: dict
        dictionary with the countries as keys and their region as values
        for region mapping

    file_dir: str
        string of the root dir where the flourish data must be saved
    
    file_name: str
        the name of the *.csv file to be created
    '''
    from os import path
    from calendar import month_abbr
    from pandas import DataFrame, concat, read_csv

    print('--------------------------')
    print('Creating files for the flourish studio slope chart')
    
    try:
        new_case = case + ' new cases'
 
        columns = ['Country/Region','Date',case]
        df['Date'] = df['Date'].transform(lambda x:x.month)

        countries = df['Country/Region'].unique()
        df_slope_chart = DataFrame()

        for country in countries:
            df_aux = df[columns].loc[df['Country/Region'] == country]
            df_aux = df_aux[df_aux.Date >= initial_month]
            total_confirmed = df_aux[case].max()

            df_aux[new_case] = df_aux.groupby('Country/Region')[case].diff().fillna(value=0)
            df_aux = df_aux.groupby(['Country/Region','Date']).sum().reset_index().drop(case,axis=1)

            df_aux[new_case] = (df_aux[new_case]*100/total_confirmed).round(1)
            df_aux[new_case] = df_aux[new_case].transform(lambda x:max(0,x))
            df_aux = df_aux.pivot(index='Country/Region',columns='Date',values=new_case)

            df_slope_chart = concat([df_slope_chart,df_aux]).fillna(value=0)

        df_slope_chart.columns = [month_abbr[i] for i in df_slope_chart.columns]
        df_region = read_csv('region_mapping.csv',index_col = 'Country/Region')
        
        #df_slope_chart = df_slope_chart.join(df_region, on = 'Country/Region', how='inner')
        
        df_slope_chart.to_csv(path.join(file_dir,file_name + '.csv'),index=False)
        print('Files created succesfully!')
    except:
        print('Process aborted! No files for flourish studio were created.')
    finally:
        print('End execution of the flourish slope chart function.')
        print('--------------------------')

In [8]:
df = data.copy()

# 1 - Racing bars chart:
initial_date = '2020-03-06'
parameters = ['Active','Confirmed','Deaths','Recovered']

flourish_racing_bars(df,parameters,initial_date,file_dir)

# ===============
# 2 - Parliament map:

seats = ['Confirmed','Active','Recovered','Deaths']
flourish_parliament_map(df,seats,region_mapping_dict,file_dir)

# ===============
# 3 - Point map:

lat = pd.read_csv('coordinates.csv',header=None,index_col=0).to_dict()[1]
long = pd.read_csv('coordinates.csv',header=None,index_col=0).to_dict()[2]
parameters = ['Confirmed','Active','Recovered','Deaths']

flourish_point_map(df,parameters,lat,long,file_dir)

# ===============
# 4 - Hierarchy chart:

cases = ['Confirmed','Active','Recovered','Deaths']
flourish_hierarchy_chart(df,cases,region_mapping_dict,file_dir)

# ===============
# 5 - Card plot:

cases = ['Confirmed','Active','Recovered','Deaths']
flourish_card_plot(df,cases,region_mapping_dict,file_dir)

# ===============
# 6 - Survey chart:

cases = ['Active','Recovered','Deaths']
flourish_survey_chart(df,cases,region_mapping_dict,file_dir)

# ===============
# 7 - Slope chart:

flourish_survey_chart(df,file_dir)

--------------------------
Creating files for the flourish racing bars chart
creating the Active cases file
creating the Confirmed cases file
creating the Deaths cases file
creating the Recovered cases file
Files created succesfully!
End execution of the flourish racing bars chart function.
--------------------------
--------------------------
Creating files for the flourish studio parliament map
Files created succesfully!
End execution of the flourish parliament map function.
--------------------------
--------------------------
Creating files for the flourish studio point map
Files created succesfully!
End execution of the flourish point map function.
--------------------------
--------------------------
Creating files for the flourish studio hierarchy chart
Files created succesfully!
End execution of the flourish hierarchy chart function.
--------------------------
